In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

torch.manual_seed(1)

import numpy as np
import pandas as pd

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

from nltk.corpus import wordnet
import json

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [5]:
import gensim
gensim_model = gensim.models.KeyedVectors.load_word2vec_format('./word2vec/data.bin', binary=True)

In [6]:
weights = gensim_model.wv.syn0
# weights.shape

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  if __name__ == '__main__':


In [9]:
weights = np.append(weights,np.zeros((1,embedding_dim)),axis=0)
# 末尾にunknown_wordを追加

In [10]:
vocab_size = weights.shape[0]
embedding_dim = weights.shape[1]

In [12]:
cuda = torch.cuda.is_available()

In [13]:
class LSTMTagger(nn.Module):
    
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.out = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        h = torch.zeros(1, 1, self.hidden_dim)
        c = torch.zeros(1, 1, self.hidden_dim)
        if cuda:
            h = h.cuda()
            c = c.cuda()
        return (h,c)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
#         print(embeds.size())

        lstm_output, self.hidden = self.lstm(
            embeds.view(len(sentence),1,-1), self.hidden)

        output = self.out(lstm_output.view(len(sentence),-1))
        output = F.tanh(output)
        return output

hidden_dim = 6
out_size = 3
model = LSTMTagger(embedding_dim, hidden_dim, vocab_size, out_size)

In [10]:
# # wordのindexを取得
# print(gensim_model.wv.vocab["'d"].index)
# # 100番目のwordを取得
# print(gensim_model.wv.index2word[100])

In [14]:
import re
import nltk
from nltk import word_tokenize

In [15]:
def prepare_sequence(seq):
    vocab = gensim_model.wv.vocab
    idxs = [vocab[w].index if w in vocab else vocab_size - 1 for w in seq]
    res = torch.tensor(idxs, dtype=torch.long)
    if cuda:
        res = res.cuda()
    return res

In [16]:
def sentence2vec(sentence,debug=False):
    w_list = word_tokenize(sentence)
    w_list = [wordnet.morphy(w) if wordnet.morphy(w) is not None else w for w in w_list]
    if debug:
        print(w_list)
    res_seq = prepare_sequence(w_list)
    return res_seq

In [18]:
json_name = './dat/loss_data_{0}.json'.format(hidden_dim)
model_name = './dat/model_data_{0}'.format(hidden_dim)

model.load_state_dict(torch.load(model_name))

In [22]:
if cuda:
    model.cuda()

In [25]:
# dataのload
data_pre = pd.read_csv('./data_preprocessed.csv')

# 2word以上のsentence
data_pre = data_pre[data_pre['words']>=2]

X_orig = data_pre['reg'].as_matrix()
Y_v = data_pre['Valence'].as_matrix()
Y_a = data_pre['Arousal'].as_matrix()
Y_d = data_pre['Dominance'].as_matrix()
Y_orig = np.c_[Y_v,Y_a,Y_d]
a=1
b=5
Y_orig = (2*(Y_orig-a)/(b-a))-1
# [-1,1]で正規化


X = X_orig
Y = Y_orig

train_size = 0.7
dev_size = 0.2
X_train, X_rest, Y_train, Y_rest = train_test_split(X, Y, test_size=1-train_size)
X_dev, X_test, Y_dev, Y_test = train_test_split(X_rest,Y_rest,test_size=1-(train_size+dev_size))


In [57]:
def make_prediction(model,df):
    tensor_list = []
    for index,row in df.iterrows():
        sentence = row['sentence']
        model.zero_grad()
        model.hidden = model.init_hidden()
        sentence_in = sentence2vec(sentence)
        y = model(sentence_in)[-1,:]
        tensor_list.append(y)
#     print(tensor_list[0])
    res = torch.stack(tensor_list)
    return res.cpu().detach().numpy()

In [60]:
prediction = make_prediction(model,data_pre)

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  from ipykernel import kernelapp as app


In [72]:
prediction = prediction*2+3

In [73]:
data_predicted = data_pre.assign(
    Valence_predicted = prediction[:,0],
    Arousal_predicted = prediction[:,1],
    Dominance_predicted = prediction[:,2]
)

In [74]:
data_predicted

,Unnamed: 0,id,sentence,Arousal,Dominance,Valence,sd.Arousal,sd.Dominance,sd.Valence,freq,reg,words,Valence_predicted,Arousal_predicted,Dominance_predicted
0,0,Acephalous-Cant-believe_4_47,I can't believe I wrote all that last year.,3.400000,3.200000,3.000000,0.800000,0.400000,0.000000,5,i can't believe i wrote all that last year,10,2.681283,3.047741,2.978069
1,1,Acephalous-Cant-believe_83_354,Because I've been grading all damn day and am ...,3.200000,3.200000,2.800000,0.400000,0.400000,0.400000,5,because i've been grading all damn day and am ...,51,3.038605,3.044772,3.072384
2,2,Acephalous-Cant-believe_355_499,"However, when I started looking through my arc...",3.000000,3.200000,3.400000,0.000000,0.400000,0.489898,5,however when i started looking through my arc...,26,2.949015,3.017131,3.042916
3,3,Acephalous-Cant-believe_500_515,What do I mean?,3.000000,3.000000,3.000000,0.000000,0.000000,0.000000,5,what do i mean?,5,2.779138,3.031124,3.002569
4,4,Acephalous-Cant-believe_517_626,The posts I consider foundational to my curren...,3.000000,3.000000,3.000000,0.000000,0.000000,0.000000,5,the posts i consider foundational to my curren...,18,2.922856,3.024922,3.037882
5,5,Acephalous-Cant-believe_627_817,"Granted, the likelihood that I'm suffering fro...",2.666667,2.833333,3.000000,0.471405,0.372678,0.000000,6,granted the likelihood that i'm suffering fro...,34,2.671841,3.084090,2.978549
6,6,Acephalous-Cant-believe_819_984,It may also be that I remember the scene of th...,3.000000,3.000000,3.000000,0.000000,0.000000,0.000000,5,it may also be that i remember the scene of th...,32,2.859691,3.091564,3.033214
7,7,Acephalous-Cant-believe_986_1160,If you have absolutely nothing better to do on...,3.000000,3.200000,3.000000,0.000000,0.400000,0.000000,5,if you have absolutely nothing better to do on...,36,3.171721,3.049678,3.109189
8,8,Acephalous-Cant-believe_1162_1229,"If you do have something better to do, though,...",3.000000,3.200000,2.800000,0.000000,0.748331,0.400000,5,if you do have something better to do though ...,14,3.053498,3.024969,3.079944
9,9,Acephalous-Cant-believe_1266_1336,Why is someone so young not having fun with fr...,3.000000,2.800000,2.600000,0.000000,0.400000,0.489898,5,why is someone so young not having fun with fr...,14,3.068133,3.018314,3.076349


the hunchbacked being who let us in held out a scraggly paw for him to slap 
[-0.33333333  0.          0.        ]
[-0.3386181   0.08111124 -0.06487487]
